In [163]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import polyline

In [164]:
df = pd.read_csv('../data/activities_small.csv', encoding='utf-8') # read is small dataset

In [165]:
df.columns.tolist()

[u'id',
 u'name',
 u'distance',
 u'moving_time',
 u'elapsed_time',
 u'total_elevation_gain',
 u'type',
 u'start_date',
 u'start_date_local',
 u'timezone',
 u'achievement_count',
 u'kudos_count',
 u'comment_count',
 u'athlete_count',
 u'photo_count',
 u'total_photo_count',
 u'trainer',
 u'commute',
 u'manual',
 u'private',
 u'flagged',
 u'average_speed',
 u'max_speed',
 u'average_watts',
 u'max_watts',
 u'weighted_average_watts',
 u'kilojoules',
 u'device_watts',
 u'has_heartrate',
 u'average_heartrate',
 u'max_heartrate',
 u'athlete_id',
 u'map_id',
 u'map_summary_polyline',
 u'start_lat',
 u'start_lng',
 u'end_lat',
 u'end_lng',
 u'state',
 u'closest_city']

In [166]:
# df = df[df['commute'] == 0] # drop all where commute = true
df.shape

(9925, 40)

In [167]:
df.head()

,id,name,distance,moving_time,elapsed_time,total_elevation_gain,type,start_date,start_date_local,timezone,...,max_heartrate,athlete_id,map_id,map_summary_polyline,start_lat,start_lng,end_lat,end_lng,state,closest_city
0,467664896,Fat Bike in the snow at GM,12557.8,4111.0,4193.0,419.4,Ride,2016-01-10 19:13:47+00:00,2016-01-10 12:13:47,America/Denver,...,NaN,7360660,a467664896,w`gqFnpx`SDzEkDYw@wBApDdBjBGlD~@~@u@pCgJdD_AdM...,39.69,-105.15,39.69,-105.15,Colorado,Lakewood
1,308543489,"05/19/2015 Fruita, CO Horse Thief Bench",15941.8,3448.0,4356.0,322.0,Ride,2015-05-20 00:41:46+00:00,2015-05-19 18:41:46,America/Denver,...,148.0,113571,a308543489,scbnFjbgwSQjMzFhGnAzMJdEgBdBWtDpA`G|NnU`AxImAx...,39.17,-108.83,39.18,-108.83,Colorado,Fruita
2,432799746,Lunch Ride,77276.3,10158.0,15286.0,962.0,Ride,2015-11-14 19:21:32+00:00,2015-11-14 12:21:32,America/Denver,...,NaN,67365,a432799746,kf`sFhaiaS}KiBsAtFyI`ByeAjyBqBjNeJt@iC_EcGsAcn...,39.98,-105.24,39.98,-105.24,Colorado,Boulder
3,404750345,Truck lot commute,19340.0,2261.0,2404.0,25.9,Ride,2015-10-02 13:59:13+00:00,2015-10-02 07:59:13,America/Denver,...,NaN,606184,a404750345,akotFz{naSHwFdJRna@wh@nGeRbOyUdE{M|@qKaAmw@rXi...,40.22,-105.27,40.16,-105.10,Colorado,Lyons
4,308543507,"05/20/2015 Fruita, CO Edge Loop",53577.9,14160.0,19563.0,1355.0,Ride,2015-05-20 15:55:46+00:00,2015-05-20 09:55:46,America/Denver,...,159.0,113571,a308543507,ez|nFrrnvSvRzu@aKhOsw@|FzDxFoKpZlMz_@qBpL}PrCk...,39.31,-108.71,39.31,-108.71,Colorado,Fruita


In [168]:
lats = df.start_lat.values

In [169]:
lngs = df.start_lng.values

In [305]:
from itertools import izip
dsts = []
for (lt, lg) in izip(lats, lngs):
    dsts.append(np.sqrt((lats - lt)**2 + (lngs - lg)**2))

In [306]:
dsts = np.array(dsts)

In [307]:
dsts.shape

(9925, 9925)

In [308]:
dsts[:4, :4]

array([[ 0.        ,  3.71655755,  0.30364453,  0.54341513],
       [ 3.71655755,  0.        ,  3.68024456,  3.7116169 ],
       [ 0.30364453,  3.68024456,  0.        ,  0.24186773],
       [ 0.54341513,  3.7116169 ,  0.24186773,  0.        ]])

In [309]:
np.argsort(dsts, axis=0)[1:11,0]

array([3835, 3760, 2508, 9735,  630, 2574, 5348, 6636, 6635, 6634])

In [310]:
np.argwhere(dsts <= 0.01)[:10]

array([[   0,    0],
       [   0,  630],
       [   0, 2508],
       [   0, 2574],
       [   0, 3760],
       [   0, 3835],
       [   0, 5348],
       [   0, 9735],
       [   1,    1],
       [   1,  299]])

In [311]:
def make_indexer(arr):
    indexer = {}
    for pair in arr:
        if pair[0] == pair[1]:
            pass
        else:
            if pair[0] in indexer:
                indexer[pair[0]].append(pair[1])
            else:
                indexer[pair[0]] = [pair[1]]
    return indexer
start_clusters = make_indexer(np.argwhere(dsts <= 0.01))

In [312]:
# start_clusters

In [313]:
poly1 = np.array(polyline.decode(df.ix[0, 'map_summary_polyline'])).flatten()

In [314]:
poly2 = np.array(polyline.decode(df.ix[630, 'map_summary_polyline'])).flatten()

In [315]:
poly1[-2:]

array([  39.69053, -105.15225])

In [316]:
poly2[-2:]

array([  39.69034, -105.15237])

In [319]:
if np.linalg.norm(np.subtract(poly1[-2:], poly2[-2:])) <= 0.01:
    if len(poly1) >= len(poly2):
        diff = len(poly1) - len(poly2)

In [329]:
poly2 = np.hstack((poly2, poly1[-diff:]))

In [331]:
poly1[-20:]

array([  39.68909, -105.1584 ,   39.68848, -105.1588 ,   39.68898,
       -105.15729,   39.68939, -105.15727,   39.68811, -105.15692,
         39.68818, -105.15641,   39.68862, -105.15632,   39.68849,
       -105.15401,   39.68957, -105.15389,   39.69053, -105.15225])

In [332]:
poly2[-20:]

array([  39.68813, -105.15648,   39.68862, -105.15629,   39.68854,
       -105.154  ,   39.68949, -105.15394,   39.69063, -105.15223,
         39.69034, -105.15237,   39.68862, -105.15632,   39.68849,
       -105.15401,   39.68957, -105.15389,   39.69053, -105.15225])

In [336]:
np.linalg.norm(np.subtract(poly1, poly2))

0.099355386366310972

In [338]:
len(poly1) / (8/2)

42